Data source: http://www.tse.com.tw/zh/page/trading/exchange/MI_5MINS_INDEX.html
1. before 2011-01-16 is a data per min
2. before 2014-02-23 is a data per 15 sec
3. before 2014-12-29 is a data per 10 sec
4. after 2014-12-29 to today is a data per 5 sec

What we need to complete this shit
1. script1: from 2004-10-15 to today
2. script2: from today keep update data, need to use another script
3. creat a DB with a table contain every columns, different date have different column lenth. (how to make it auto variable with data change?)  (idea: for script1, just drop the fix numbers columns. Then, as script2 insert data, just insert the columns which has time column)  

In [2]:
# -*-coding: utf-8 -*-
import sys
import datetime as dt
import requests
import os
import pandas as pd
import sqlite3
import csv
import time


        
def date_trans(start1,end1):   # date transformat fail

    date_int={'year1':int(start1[:4]),'month1':int(start1[4:6]),'day1':int(start1[6:]),
              'year2':int(end1[:4]),'month2':int(end1[4:6]),'day2':int(end1[6:])}
    
    start_date = dt.datetime(date_int['year1'],date_int['month1'],date_int['day1'])
    end_date = dt.datetime(date_int['year2'],date_int['month2'],date_int['day2'])
    totaldays = (end_date - start_date).days + 1
    return start_date,totaldays


def get_content(start_date,totaldays):
    with sqlite3.connect('testDB') as DB:
        for daynumber in range(totaldays):
            date1=(start_date + dt.timedelta(days = daynumber)).date()
            five_sec_url='http://www.twse.com.tw/exchangeReport/MI_5MINS?response=csv&date=%s'%str(date1).replace('-','')
            r = requests.get(five_sec_url)
            try:
                con = r.text.split('\n')
                res = delIntro(con)
                insertDate(res,str(date1))
                insertData(res,DB)
            except:
                pass
            time.sleep(2)

    
def delIntro(con):    # delete the introduction of content
    del con[0]
    del con[con.index('"說明:"\r'.encode('utf-8').decode('utf-8', 'ignore')):]  
    res = [[con[i].strip('=').strip() for i in range(len(con))][j][1:-2].replace(',','').split('\"\"') for j in range(len(con))]
    return res

    
def insertDate(res,date1):    # insert date to first column 
    res[0].insert(0,'日期')
    for i in range(len(res)-1):
        res[i+1].insert(0,date1)

        
def createTable():
    with sqlite3.connect('testDB') as DB:
        c=DB.cursor()
        createTable='''create table if not exists fiveSec (日期 date, 時間 time,累積委託買進筆數 int,  累積委託買進數量 int, 累積委託賣出筆數 int, 累積委託賣出數量 int, 累積成交筆數 int, 累積成交數量 int, 累積成交金額 int);'''
        c.execute(createTable)
        
        
def insertData(res,DB):
#     with sqlite3.connect('testDB') as DB:    # here should change, everytime reconnect is not a good way
        c=DB.cursor()
        c.executemany('insert into fiveSec values(?,?,?,?,?,?,?,?,?)',res[1:])
        
def selectData():
    with sqlite3.connect('testDB') as DB:
        df = pd.read_sql_query('select * from fiveSec',DB)
        print(df)

        
def iftttRemind(text1,value1,value2):
    iftttCode='clJBzxCpWf6A8RIS-Ia6C5'
    url_ifttt = 'https://maker.ifttt.com/trigger/stockline/with/key/%s?value1=%s/value2=%f/value3=%s'%(iftttCode,text1,value1,value2)
    requests.get(url_ifttt)


# main   
if __name__ =='__main__': 
    starttime = time.time()
    createTable()  
    start_date,totaldays=date_trans('20180801','20180811')
#         start_date,totaldays=date_trans(sys.args[0],sys.args[1])
    get_content(start_date,totaldays)
#     selectData()
    cost_time=time.time()-starttime
    print('cost time:',cost_time)
    iftttRemind('cost time:', cost_time,'OK')

cost time: 45.710230350494385
